# Iteration 5: organise parameters

## Plain English Summary

Before we continue with adding in model logic and result collection the organisation of the model is modified so that parameters are separated from model logic.  This modification will allow us to test the model more fully. 

The same tests can be applied here as iteration 1, 2, 3, and 4. These should now be easier as parameters should not be hardcoded and are instead accessible via the Experiment class.  Note that the tests are modified to work with the new Experiment class

## Notebook imports

In [1]:
from sim_tools.distributions import Lognormal

## LLM Prompt

::::{admonition} LLM Prompt
:::{include} inputs/05_iteration.md
:::
::::

## Date and time of input to LLM

> 1000 5th March 2024

## Observations on the LLM Output

* LLM has correctly created an `Experiment` class and passes correct default values. These are used by the `AcuteStrokeUnit` methods.
* Script to run the example uses the `Experiment` class correctly.

## Sources reported by the LLM

* None given - Perplexity appears to have stopped providing them.

## LLM Output

In [3]:
import simpy
import numpy as np

# External function to convert Lognormal moments to Normal moments
def normal_moments_from_lognormal(mean, std_dev):
    phi = np.sqrt(std_dev ** 2 + mean ** 2)
    mu = np.log(mean ** 2 / np.sqrt(std_dev ** 2 + mean ** 2))
    sigma = np.sqrt(np.log(phi ** 2 / mean ** 2))
    return mu, sigma

# Define the Experiment class to store model parameters
class Experiment:
    def __init__(self, stroke_mean=1.2, tia_mean=9.3, neuro_mean=3.6, other_mean=3.2,
                 rehab_mean=7.4, rehab_std_dev=8.6, esd_mean=4.6, esd_std_dev=4.8,
                 other_dest_mean=7.0, other_dest_std_dev=8.7,
                 tia_dest_mean=1.8, tia_dest_std_dev=5.0,
                 neuro_dest_mean=4.0, neuro_dest_std_dev=5.0,
                 other_dest_mean_2=3.8, other_dest_std_dev_2=5.2,
                 run_length=1825):  # 5 years in days
        self.stroke_interarrival_mean = stroke_mean
        self.tia_interarrival_mean = tia_mean
        self.neuro_interarrival_mean = neuro_mean
        self.other_interarrival_mean = other_mean
        self.rehab_mean = rehab_mean
        self.rehab_std_dev = rehab_std_dev
        self.esd_mean = esd_mean
        self.esd_std_dev = esd_std_dev
        self.other_dest_mean = other_dest_mean
        self.other_dest_std_dev = other_dest_std_dev
        self.tia_dest_mean = tia_dest_mean
        self.tia_dest_std_dev = tia_dest_std_dev
        self.neuro_dest_mean = neuro_dest_mean
        self.neuro_dest_std_dev = neuro_dest_std_dev
        self.other_dest_mean_2 = other_dest_mean_2
        self.other_dest_std_dev_2 = other_dest_std_dev_2
        self.run_length = run_length

# Define the AcuteStrokeUnit class with patient generators using Experiment parameters
class AcuteStrokeUnit:
    def __init__(self, env, experiment):
        self.env = env
        self.experiment = experiment
        self.patient_count = 0

    def stroke_acute_treatment(self, patient_type):
        if patient_type == "Rehab":
            mu, sigma = normal_moments_from_lognormal(self.experiment.rehab_mean, self.experiment.rehab_std_dev)
            length_of_stay = np.random.lognormal(mean=mu, sigma=sigma)
        elif patient_type == "ESD":
            mu, sigma = normal_moments_from_lognormal(self.experiment.esd_mean, self.experiment.esd_std_dev)
            length_of_stay = np.random.lognormal(mean=mu, sigma=sigma)
        else:
            mu, sigma = normal_moments_from_lognormal(self.experiment.other_dest_mean, self.experiment.other_dest_std_dev)
            length_of_stay = np.random.lognormal(mean=mu, sigma=sigma)
        
        yield self.env.timeout(length_of_stay)
        print(f"Stroke patient {self.patient_count} finished treatment at {self.env.now} days")

    def tia_acute_treatment(self):
        mu, sigma = normal_moments_from_lognormal(self.experiment.tia_dest_mean, self.experiment.tia_dest_std_dev)
        length_of_stay = np.random.lognormal(mean=mu, sigma=sigma)
        yield self.env.timeout(length_of_stay)
        print(f"TIA patient {self.patient_count} finished treatment at {self.env.now} days")

    def neuro_acute_treatment(self):
        mu, sigma = normal_moments_from_lognormal(self.experiment.neuro_dest_mean, self.experiment.neuro_dest_std_dev)
        length_of_stay = np.random.lognormal(mean=mu, sigma=sigma)
        yield self.env.timeout(length_of_stay)
        print(f"Complex Neuro patient {self.patient_count} finished treatment at {self.env.now} days")

    def other_acute_treatment(self):
        mu, sigma = normal_moments_from_lognormal(self.experiment.other_dest_mean_2, self.experiment.other_dest_std_dev_2)
        length_of_stay = np.random.lognormal(mean=mu, sigma=sigma)
        yield self.env.timeout(length_of_stay)
        print(f"Other patient {self.patient_count} finished treatment at {self.env.now} days")

    def stroke_patient_generator(self):
        while True:
            interarrival_time = np.random.exponential(self.experiment.stroke_interarrival_mean)
            yield self.env.timeout(interarrival_time)
            self.patient_count += 1
            post_asu_destination = np.random.choice(["Rehab", "ESD", "Other"], p=[0.24, 0.13, 0.63])
            print(f"Stroke patient {self.patient_count} arrived at {self.env.now} days and will go to {post_asu_destination}")
            self.env.process(self.stroke_acute_treatment(post_asu_destination))

    def tia_patient_generator(self):
        while True:
            interarrival_time = np.random.exponential(self.experiment.tia_interarrival_mean)
            yield self.env.timeout(interarrival_time)
            self.patient_count += 1
            print(f"TIA patient {self.patient_count} arrived at {self.env.now} days")
            self.env.process(self.tia_acute_treatment())

    def neuro_patient_generator(self):
        while True:
            interarrival_time = np.random.exponential(self.experiment.neuro_interarrival_mean)
            yield self.env.timeout(interarrival_time)
            self.patient_count += 1
            print(f"Complex Neuro patient {self.patient_count} arrived at {self.env.now} days")
            self.env.process(self.neuro_acute_treatment())

    def other_patient_generator(self):
        while True:
            interarrival_time = np.random.exponential(self.experiment.other_interarrival_mean)
            yield self.env.timeout(interarrival_time)
            self.patient_count += 1
            print(f"Other patient {self.patient_count} arrived at {self.env.now} days")
            self.env.process(self.other_acute_treatment())

# Create the simulation environment and Experiment instance with default parameters
env = simpy.Environment()
default_experiment_params = Experiment()
acu_experiment = AcuteStrokeUnit(env, default_experiment_params)

# Start the patient generators for each type of patient
env.process(acu_experiment.stroke_patient_generator())
env.process(acu_experiment.tia_patient_generator())
env.process(acu_experiment.neuro_patient_generator())
env.process(acu_experiment.other_patient_generator())

# Run the simulation until the specified run length in the Experiment parameters
env.run(until=default_experiment_params.run_length)


Complex Neuro patient 1 arrived at 2.8171732718641502 days
Stroke patient 2 arrived at 3.0671000155809307 days and will go to Rehab
TIA patient 3 arrived at 3.150052509583502 days
TIA patient 3 finished treatment at 3.3300172506344112 days
Other patient 4 arrived at 3.480939360626922 days
Stroke patient 5 arrived at 4.553737966233015 days and will go to Other
Stroke patient 6 arrived at 4.991062055557084 days and will go to ESD
Stroke patient 6 finished treatment at 5.056044075130818 days
Stroke patient 7 arrived at 6.348723282735784 days and will go to Rehab
Stroke patient 8 arrived at 6.678333927873286 days and will go to Other
Stroke patient 9 arrived at 6.861502499127046 days and will go to Other
Stroke patient 9 finished treatment at 7.46207083063959 days
Stroke patient 9 finished treatment at 8.553266312470306 days
Stroke patient 10 arrived at 8.873926492487866 days and will go to Other
Stroke patient 10 finished treatment at 9.962509340094234 days
Stroke patient 11 arrived at 10

## Testing
* **Testing lognormal function**: Test that lognomal function correctly calculates the moments of the underlying normal dist.
    * Expected result: The LLM generated function and `sim-tools.distributions.Lognormal` perform the same conversion.
    * Actual results (PASS):  The functions generate the same values.
* **Extreme value test 1**: TIA, Complex Neuro, Other, have their inter-arrival time set to $M$ a very large number
    * Expected result: The only type of patient to arrive to the model is "Stroke"
    * Actual result (PASS): The only type of patient to arrive to the model is "Accident and Emergency."
* **Extreme value test 2**: All patient types have have their inter-arrival time set to $M$ a very large number
    * Expected result: No patients arrive to the model.
    * Actual result (PASS): No patients arrive to the model.
* **Different run length**: The run length of the model is set to 10 days.
    * Expected result: The model should runs no longer than 10 simulated time periods
    * Actual result (PASS): The model runs no longer than 10 simulated time periods
* **Extreme value test 3**: All patient types have have their mean length of stay time set to $M$ a very large number
    * Expected result: No patients depart the model (only arrivals seen in log)
    * Actual Results (PASS): No patients depart the model.
* **Extreme value test 4**: All patient types apart from stroke->rehab have have their mean length of stay time set to $M$ a very large number
    * Expected result: Only stroke patients depart the model (seen in log)
    * Actual Results (PASS): Only stroke patient depart the model.

### Lognormal test

Test that lognomal function correctly calculates the moments of the underlying normal dist.

**Result: PASS**

In [4]:
def test_lognormal_moments(mean, std):
    '''
    Test that lognomal function correctly calculates 
    the moments of the underlying normal dist.

    Params:
    ------
    mean: float
        mean of the lognormal distribution

    std: float
        st dev of the lognormal distribution

    Returns:
    -------
    bool
    '''
   
    # Lognormal class from sim-tools.
    expected_moments = Lognormal(mean, std)
    print(expected_moments.mu, expected_moments.sigma)
    
    ## llm code
    llm_mu, llm_sigma = normal_moments_from_lognormal(mean, std)
    print(llm_mu, llm_sigma)
    
    return (llm_mu, llm_sigma) == (expected_moments.mu, expected_moments.sigma)

In [5]:
test_lognormal_moments(128.79, 267.51)

4.022977459259615 1.2924439729790718
4.022977459259615 1.2924439729790718


True

### Extreme value test 1

The model uses hard coded parameters.  The most simple way to modify the code for this test was to change hard coded parameters for $M$

**Result: PASS**

In [6]:
M = 10_000_000

In [7]:
def test_1(large_number):
    '''
    TIA, Complex Neuro, Other, have their inter-arrival 
    time is set to $M$ a very large number
    
    Expected result: The only type of patient to arrive to the model 
    is "Stroke". This is seen in the event log

    Params:
    -------
    large_number: int
        M a very large number 

    Returns:
    --------
    int: the number of patients that arrived to the model.
    '''
    # Create the simulation environment
    env = simpy.Environment()
    
    # Initialize the Acute Stroke Unit model#
    # set ASU arrival rates
    default_experiment_params = Experiment(
        tia_mean=large_number,
        neuro_mean=large_number,
        other_mean=large_number
    )

    acu_experiment = AcuteStrokeUnit(env, default_experiment_params)
    
    # Start the patient generators for each type of patient
    env.process(acu_experiment.stroke_patient_generator())
    env.process(acu_experiment.tia_patient_generator())
    env.process(acu_experiment.neuro_patient_generator())
    env.process(acu_experiment.other_patient_generator())
    
    # Run the simulation until the specified run length in the Experiment parameters
    env.run(until=default_experiment_params.run_length)

    return acu_experiment.patient_count

In [8]:
test_1(M)

Stroke patient 1 arrived at 1.450119040332075 days and will go to Other
Stroke patient 2 arrived at 2.865009544704182 days and will go to Other
Stroke patient 3 arrived at 3.7855241529225663 days and will go to Other
Stroke patient 4 arrived at 5.608115699054348 days and will go to Other
Stroke patient 5 arrived at 6.203180021207933 days and will go to Other
Stroke patient 6 arrived at 6.340114702676991 days and will go to Other
Stroke patient 7 arrived at 6.340526005439017 days and will go to Rehab
Stroke patient 8 arrived at 7.293656264419474 days and will go to Rehab
Stroke patient 9 arrived at 9.998790201921727 days and will go to Rehab
Stroke patient 9 finished treatment at 10.10931594546162 days
Stroke patient 10 arrived at 10.80415987386928 days and will go to Rehab
Stroke patient 10 finished treatment at 11.371998659479688 days
Stroke patient 10 finished treatment at 11.41163484053474 days
Stroke patient 11 arrived at 11.968667953956714 days and will go to Other
Stroke patient 

1452

### Extreme value test 2

* An experiment is setup so that all patient types have a mean inter-arrival rate of $M$

**Result: PASS**

In [9]:
def test_2(large_number):
    '''
    All patient types have have their inter-arrival time is 
    set to $M$ a very large number
    
    Expected result: No patients arrive to the model.
    
    Params:
    -------
    large_number: int
        M a very large number 

    Returns:
    --------
    int: the number of patients that arrived to the model.
    '''
    # Create the simulation environment
    env = simpy.Environment()
    
    # Initialize the Acute Stroke Unit model#
    # set ASU arrival rates
    default_experiment_params = Experiment(
        stroke_mean=large_number,
        tia_mean=large_number,
        neuro_mean=large_number,
        other_mean=large_number
    )

    acu_experiment = AcuteStrokeUnit(env, default_experiment_params)
    
    # Start the patient generators for each type of patient
    env.process(acu_experiment.stroke_patient_generator())
    env.process(acu_experiment.tia_patient_generator())
    env.process(acu_experiment.neuro_patient_generator())
    env.process(acu_experiment.other_patient_generator())
    
    # Run the simulation until the specified run length in the Experiment parameters
    env.run(until=default_experiment_params.run_length)

    return acu_experiment.patient_count

In [10]:
test_2(M)

0

### Different run length test

The model has again been modified to use the original code and parameters generated by the LLM.

In [11]:
TEN_DAYS = 10

In [12]:
def test_3(run_length):
    '''
    Test that the run length of the model can be modified.
    
    Expected result: The model should runs no longer than the new run length.

    Params:
    ------
    run_length: float
        Run length of the model in days.

    Returns:
    -------
    float
        Simulation end time.
    '''
    # Create the simulation environment
    env = simpy.Environment()
    
    # Initialize the Acute Stroke Unit model#
    # set ASU arrival rates
    default_experiment_params = Experiment(
        run_length=run_length
    )

    acu_experiment = AcuteStrokeUnit(env, default_experiment_params)
    
    # Start the patient generators for each type of patient
    env.process(acu_experiment.stroke_patient_generator())
    env.process(acu_experiment.tia_patient_generator())
    env.process(acu_experiment.neuro_patient_generator())
    env.process(acu_experiment.other_patient_generator())
    
    # Run the simulation until the specified run length in the Experiment parameters
    env.run(until=default_experiment_params.run_length)

    return env.now

In [13]:
test_3(TEN_DAYS)

Other patient 1 arrived at 0.11807841115047267 days
Stroke patient 2 arrived at 0.5281712053546487 days and will go to Rehab
Stroke patient 3 arrived at 0.6276326912965233 days and will go to Other
Other patient 4 arrived at 1.2490375764829236 days
Other patient 5 arrived at 1.527593301330492 days
Other patient 5 finished treatment at 1.5981260197280593 days
Stroke patient 6 arrived at 3.1167608438571484 days and will go to ESD
Stroke patient 7 arrived at 3.418158370077752 days and will go to Other
Stroke patient 8 arrived at 3.4600358400098554 days and will go to Other
Stroke patient 8 finished treatment at 3.5103044204186515 days
Stroke patient 9 arrived at 3.5142202589361258 days and will go to ESD
Stroke patient 10 arrived at 3.5574090718152487 days and will go to Other
Other patient 10 finished treatment at 3.6217870796603293 days
Stroke patient 11 arrived at 4.721389556636106 days and will go to Other
Stroke patient 11 finished treatment at 5.926579805990509 days
TIA patient 12 a

10

### Extreme value test 3

* An experiment is setup so that all patient types have a mean length of stay on the ASU of $M$

**Result: PASS**

In [14]:
def test_4(large_number):
    '''
    All patient types have have their mean length 
    of stay time set to $M$ a very large number
    
    Expected result: No patients depart the model 
    (only arrivals seen in log)

    Params:
    -------
    large_number: int
        M a very large number 
    '''
    # Create the simulation environment
    env = simpy.Environment()
    
    # Initialize the Acute Stroke Unit model#
    # set ASU arrival rates
    default_experiment_params = Experiment(
        rehab_mean=M,  # stroke->rehab
        esd_mean=M,    # stroke-> ESD
        other_dest_mean=M, # stroke->other
        tia_dest_mean=M,   # TIA patients
        neuro_dest_mean=M, # complex neuro patients
        other_dest_mean_2=M, # other (med outlier) patients
    )

    acu_experiment = AcuteStrokeUnit(env, default_experiment_params)
    
    # Start the patient generators for each type of patient
    env.process(acu_experiment.stroke_patient_generator())
    env.process(acu_experiment.tia_patient_generator())
    env.process(acu_experiment.neuro_patient_generator())
    env.process(acu_experiment.other_patient_generator())
    
    # Run the simulation until the specified run length in the Experiment parameters
    env.run(until=default_experiment_params.run_length)

In [15]:
test_4(M)

Stroke patient 1 arrived at 0.4848621298359747 days and will go to Other
Stroke patient 2 arrived at 1.668442159553314 days and will go to ESD
Other patient 3 arrived at 2.6424686767959145 days
Complex Neuro patient 4 arrived at 2.886609879119907 days
Stroke patient 5 arrived at 3.0102552326870153 days and will go to Other
Stroke patient 6 arrived at 3.5768020407000716 days and will go to Other
Other patient 7 arrived at 3.82618778855732 days
Stroke patient 8 arrived at 5.382769551536176 days and will go to Rehab
Stroke patient 9 arrived at 5.459519632820424 days and will go to Other
Other patient 10 arrived at 5.588112237190039 days
Stroke patient 11 arrived at 6.274432015754469 days and will go to Other
Stroke patient 12 arrived at 8.532511172713072 days and will go to Rehab
Other patient 13 arrived at 8.951409130667413 days
Complex Neuro patient 14 arrived at 9.812632424980197 days
Stroke patient 15 arrived at 10.73747349473361 days and will go to Other
Stroke patient 16 arrived at 

### Extreme value test 4

* An experiment is setup so that all patient types other than stroke->rehab have a mean length of stay on the ASU of $M$

**Result: PASS**

In [16]:
def test_5(large_number):
    '''
    All patient types apart from stroke-rehab patients
    have have their mean length 
    of stay time set to $M$ a very large number
    
    Expected result: Only stroke patients depart the
    model.
    (only arrivals seen in log)

    Params:
    -------
    large_number: int
        M a very large number 
    '''
    # Create the simulation environment
    env = simpy.Environment()
    
    # Initialize the Acute Stroke Unit model#
    # set ASU arrival rates
    default_experiment_params = Experiment(
        rehab_mean=7.4,  # stroke->rehab
        esd_mean=M,    # stroke-> ESD
        other_dest_mean=M, # stroke->other
        tia_dest_mean=M,   # TIA patients
        neuro_dest_mean=M, # complex neuro patients
        other_dest_mean_2=M, # other (med outlier) patients
    )

    acu_experiment = AcuteStrokeUnit(env, default_experiment_params)
    
    # Start the patient generators for each type of patient
    env.process(acu_experiment.stroke_patient_generator())
    env.process(acu_experiment.tia_patient_generator())
    env.process(acu_experiment.neuro_patient_generator())
    env.process(acu_experiment.other_patient_generator())
    
    # Run the simulation until the specified run length in the Experiment parameters
    env.run(until=default_experiment_params.run_length)

    print(acu_experiment.patient_count)

In [17]:
test_5(M)

Stroke patient 1 arrived at 0.21247922692938165 days and will go to Other
Other patient 2 arrived at 0.3137794142010715 days
Stroke patient 3 arrived at 0.39748302959075843 days and will go to ESD
Stroke patient 4 arrived at 0.686424157669277 days and will go to Other
Other patient 5 arrived at 0.9945858749892584 days
Stroke patient 6 arrived at 1.4563348266794525 days and will go to Other
Stroke patient 7 arrived at 2.8878671310346107 days and will go to Rehab
Stroke patient 8 arrived at 3.416254699155784 days and will go to Other
Stroke patient 9 arrived at 3.935711860206891 days and will go to Other
Complex Neuro patient 10 arrived at 3.963073563868425 days
Complex Neuro patient 11 arrived at 4.694979383007168 days
Stroke patient 12 arrived at 4.784141576597734 days and will go to Rehab
Complex Neuro patient 13 arrived at 4.7842356100352 days
Other patient 14 arrived at 4.839844564265212 days
Stroke patient 14 finished treatment at 5.256788486068863 days
Stroke patient 15 arrived at